In [1]:
import random
import numpy as np
import gym

# załadowanie środowiska
env = gym.make("FrozenLake-v0", is_slippery=False).env
# reset środowiska do losowego stanu
env.reset()

# tworzenie QTable
action_size = env.action_space.n # ilość akcji (kolumny)
state_size = env.observation_space.n # ilość stanów (wiersze)
qtable = np.zeros((state_size, action_size)) # inicjalizacja qtable zerami
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [2]:
# Parametry
episodes = 10000
lr = 0.8
steps = 99 # maksymalna liczba kroków na epokę
gamma = 0.95 # współczynnik obniżenia

# Parametry eksploracji 
eps = 1.0 # współczynnik eksploracji
max_eps_prob = 1.0 # prawdopodobieństwo eksploracji na starcie
min_eps_prob = 0.01 # najmniejsze prawdopodobieństwo eksploracji
decay_rate = 0.005 # szybkość zaniku wykładniczego

In [3]:
# Lista nagród
rewards = []
# Pętla ucząca
for episode in range(episodes):
  state = env.reset() # reset gry - początkowe ustawienie
  step = 0
  done = False
  total_rewards = 0
  for step in range(steps):
    # Wybranie akcji w obecnym stanie świata (s)
    exp_exp_tradeoff = random.uniform(0,1) # Losowa wartość
    # Jeżeli wylosowana wartość > eps, to wykonuj eksploitację 
    # (wzięcie największej wartości Q dla danego stanu)
    if exp_exp_tradeoff > eps:
      action = np.argmax(qtable[state,:]) # wybranie najwyżej wartości z wiersza
    else:
      action = env.action_space.sample() # eksploracja - wybierz losową akcję
    # Dla wybranej akcji policz stan i nagrodę
    new_state, reward, done, info = env.step(action)
    # Równanie Bellmana - aktualizacja wartości Q w qtable:
    qtable[state, action] = qtable[state, action] \
     + lr * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
    # licznik nagród
    total_rewards += reward
    # aktualizacja stanu
    state = new_state
    # Przedwczesny koniec, porażka w grze:
    if done == True:
      break
  # Redukcja epsilonu, aby z każdą sukcesywną epoką mieć mniej eksploracji,
  # a więcej eksploitacji
  eps = min_eps_prob + (max_eps_prob - min_eps_prob) * np.exp(-decay_rate
                                                              * episode)
  rewards.append(total_rewards)
print("Training finished.\n Average rewards per episode: " + str(sum(rewards)/episodes)) 
print(qtable)

Training finished.
 Average rewards per episode: 0.9661
[[0.73509189 0.77378094 0.6983373  0.73509189]
 [0.73509189 0.         0.66183271 0.69752098]
 [0.69833177 0.49165312 0.40141551 0.45412746]
 [0.63584666 0.         0.         0.        ]
 [0.77378094 0.81450625 0.         0.73509189]
 [0.         0.         0.         0.        ]
 [0.         0.9025     0.         0.63667347]
 [0.         0.         0.         0.        ]
 [0.81450625 0.         0.857375   0.77378094]
 [0.81450625 0.9025     0.9025     0.        ]
 [0.857375   0.95       0.         0.857375  ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.9025     0.95       0.857375  ]
 [0.9025     0.95       1.         0.9025    ]
 [0.         0.         0.         0.        ]]


In [7]:
# Agent grający w Frozen Lake
env.reset()
episodes, total_epochs = 100, 0
rewards = []
for episode in range(episodes):
  state = env.reset()
  step, total_rewards = 0, 0
  done = False

  for step in range(steps):
    # Wybierz akcję (indeks qtable) z najwyższą wartość nagrody dla tego stanu
    action = np.argmax(qtable[state,:])
    new_state, reward, done, info = env.step(action)
    total_rewards += reward
    if done:
      # ostatni stan - agent dotarł do celu lub wpadł do dziury
      total_epochs += step
      break
    state = new_state
  rewards.append(total_rewards)

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs/episodes}")
print(f"Average rewards per episode: {sum(rewards)/episodes}")

Results after 100 episodes:
Average timesteps per episode: 5.0
Average rewards per episode: 1.0


In [8]:
'''
  Wizualizacja
'''

state = env.reset()
done = False
while not done:
  env.render()
  action = np.argmax(qtable[state])
  state, reward, done, info = env.step(action)
env.render()


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
